In [1]:
import tensorflow as tf
import numpy as np
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sessConfig=tf.ConfigProto(gpu_options=gpu_options)

In [26]:

class BellmanTF():

    def recursiveMRPOperator(self, V, P, R, gamma):
        """Don't call it more than once over the same set of variables. Evaluate the operator whenever needed.
        
        Arguments:
            V {np.variable} -- matrix of shape [n,1] where n is the number of states
            P {np.variable} -- matrix of [n,n] where each row is s and each column is s'
            R {np.variable} -- same as the shape of V
            gamma{np.constant}
        """

        dnom = tf.subtract(1, tf.mul(P, gamma) )
        invDnom = tf.matrix_inverse(dnom)
        return V.assign(tf.matmul(invDnom, R))


    def createRowVec( self, n:int, initialStateVals=0 ):
        return tf.Variable(tf.fill( [n, 1], initialStateVals ))
    

    def createRowVecfromNumpyVector( self, npV ):
        return tf.Variable( tf.reshape(npV, [-1,1]) )


    def createRowVecfromNumpyMatrix( self, npV ):
        return tf.Variable( npV )

In [27]:
belman = Bellman()

In [8]:
mat = tf.fill([2,4], 0)
with tf.Session( config=sessConfig ) as sess:
    out = sess.run(mat)
    print( out )

[[0 0 0 0]
 [0 0 0 0]]


In [9]:
npV = np.arange(5
               )
npV

array([0, 1, 2, 3, 4])

In [13]:
bellV = tf.Variable( npV )
with tf.Session( config=sessConfig ) as sess:
    sess.run(tf.global_variables_initializer())
    out = sess.run(bellV)
    print( out )

[0 1 2 3 4]


In [116]:

class RLUtilsTF():


    def createRowVec( self, n:int, initialStateVals=0., type=None, name=None ):

        if type is None:
            return tf.Variable(tf.fill( [n, 1], initialStateVals ), name=name )
        else:
            return tf.Variable(
                        tf.cast( tf.fill( [n, 1], initialStateVals ), name=name , dtype=type )
            )
    

    def createRowVecfromNumpyVector( self, npV:np.ndarray, type=None, name=None ):

        if type is not None:
            npV = npV.astype( type )
        
        return tf.Variable( tf.reshape(npV, [-1,1]), name=name )

    def createRowVecfromNumpyMatrix( self, npV, type=None, name=None ):

        if type is not None:
            npV = npV.astype( type )
            
        return tf.Variable( npV, name=name  )
    
    def createRandomProbabiltyMatrix( self, n ):

        with tf.name_scope( 'rlUtilsP' ):

            vec = tf.get_variable("temp_vec", shape=[n], initializer=tf.zeros_initializer() )
            randomizeVec = vec.assign( tf.random_uniform( shape=[n], seed=39 ) )
            softmaxVec = tf.divide( randomizeVec, tf.reduce_sum( randomizeVec ) )

            ta = tf.TensorArray( dtype=tf.float32, size=n )

            loopVars = ( 0, ta )

            condition = lambda i, _: i < n

            body = lambda i, ta: ( i + 1, ta.write(i, softmaxVec ) )

            m, ta_final = tf.while_loop( condition, body, loopVars )

            return ta_final.stack()

    

In [117]:
rlUtils = RLUtilsTF()


In [119]:
with tf.variable_scope( 'test'):
    b1 = rlUtils.createRowVec(5, type=tf.int32)
    b2 = tf.random_uniform( [5], seed=39 )
    a = [1,2]
    b3 = rlUtils.createRandomProbabiltyMatrix(5)

with tf.Session( config=sessConfig ) as sess:
    sess.run(tf.global_variables_initializer())
    b1_out, b2_out, b3_out = sess.run([b1,b2,b3])
    print( b1_out )
    print(b2_out)
    print(b3_out)


[[0]
 [0]
 [0]
 [0]
 [0]]
[0.9911251  0.5737276  0.6360707  0.15947068 0.9381956 ]
[[0.30046934 0.17393118 0.19283111 0.04834511 0.28442323]
 [0.30046934 0.17393118 0.19283111 0.04834511 0.28442323]
 [0.30046934 0.17393118 0.19283111 0.04834511 0.28442323]
 [0.30046934 0.17393118 0.19283111 0.04834511 0.28442323]
 [0.30046934 0.17393118 0.19283111 0.04834511 0.28442323]]


In [109]:

V = rlUtils.createRowVec(5, name='V')
R = tf.random_normal([5,1], mean = 0, stddev=1.0, seed=39, name='reward')
P

NameError: name 'P' is not defined

In [ ]:
a = [1,2]
b = tf.Variable(a)